# 📊 Capstone Project: Dynamic Pricing for Urban Parking Lots
# 👨‍💻 Summer Analytics 2025 — Consulting & Analytics Club × Pathway
# 🧠 By: Nishant Roy


In this notebook, we will design and simulate a dynamic pricing model for 14 urban parking lots using:
1. Real-time data streams
2. ML models built from scratch using only NumPy and Pandas
3. Real-time simulation with Pathway
4. Visualizations using Bokeh

This project is divided into:
- 📈 Model 1: Baseline Linear Pricing
- 📈 Model 2: Demand-Based Pricing
- 🧠 Model 3: Competitive Pricing (Optional)
- 🔁 Real-time Simulation using Pathway
- 📉 Bokeh Visualization




# Install Pathway

In [20]:
# ⚙️ Install the Pathway real-time framework
!pip install pathway


# Essential Library Imports



In [21]:
# 📦 Import necessary libraries
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt

# 📈 For visualization
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
output_notebook()

# 🌀 For real-time data simulation (Pathway)
import pathway as pw


# Load and Preview the Dataset

In [22]:
# 📥 Load the dataset from file
import pandas as pd

lot_info = pd.read_csv('https://raw.githubusercontent.com/nishantroy449/UrbanParking-DynamicPricing/main/dataset.csv')

# 👀 Show the first few rows
lot_info.head()


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


# Check Column Names and Data Types

In [23]:
# 🔍 Check column names and data types
print("📄 Column Names:\n", lot_info.columns.tolist())
print("\nℹ️ DataFrame Info:\n")
lot_info.info()


📄 Column Names:
 ['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude', 'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime']

ℹ️ DataFrame Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18368 entries, 0 to 18367
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      18368 non-null  int64  
 1   SystemCodeNumber        18368 non-null  object 
 2   Capacity                18368 non-null  int64  
 3   Latitude                18368 non-null  float64
 4   Longitude               18368 non-null  float64
 5   Occupancy               18368 non-null  int64  
 6   VehicleType             18368 non-null  object 
 7   TrafficConditionNearby  18368 non-null  object 
 8   QueueLength             18368 non-null  int64  
 9   IsSpecialDay            18368 non-null  int64  
 10  LastUpdatedDate         1

# Create a Proper Timestamp Column

In [24]:
# ⏰ Combine date and time into a single timestamp column
lot_info['timestamp'] = pd.to_datetime(
    lot_info['LastUpdatedDate'] + ' ' + lot_info['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S',
    dayfirst=True, errors='coerce'  # fallback to NaT if broken
)

# ✅ Preview result
lot_info[['LastUpdatedDate', 'LastUpdatedTime', 'timestamp']].head()


,LastUpdatedDate,LastUpdatedTime,timestamp
0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,04-10-2016,08:25:00,2016-10-04 08:25:00
2,04-10-2016,08:59:00,2016-10-04 08:59:00
3,04-10-2016,09:32:00,2016-10-04 09:32:00
4,04-10-2016,09:59:00,2016-10-04 09:59:00


## ✅ Model 1: Baseline Linear Pricing — Summary

We implemented a simple pricing engine where:

- The price increases **linearly** as occupancy increases:
  
  \[
  \text{Price}_{t+1} = \text{Price}_t + \alpha \cdot \left( \frac{\text{Occupancy}}{\text{Capacity}} \right)
  \]

- The price starts at **$10**, with bounds of **$5 to $20**.
- It smoothly reacts to crowding in each lot over time.
- Each parking lot maintains its own pricing sequence.
- Visualization using Bokeh shows how price evolves in real-time.

This model acts as our **baseline reference** and will now be upgraded to include demand-related features in **Model 2** 🚗📊

---



# Define Baseline Pricing Parameters

In [25]:
# 🎯 Baseline Model Configuration
BASE_PRICE = 10.0     # Initial price in dollars
ALPHA = 2.0           # Sensitivity to occupancy (tunable)

# 💵 Price limits to keep pricing realistic
MIN_PRICE = 5.0
MAX_PRICE = 20.0


# Define Baseline Pricing Function

In [26]:
# 📈 Linear Pricing Function based on occupancy

def baseline_price(prev_price, occupancy, capacity):
    """
    Calculates next price using linear increase with occupancy ratio.

    Parameters:
    - prev_price: price at previous timestamp
    - occupancy: number of vehicles currently parked
    - capacity: total number of spots in the lot

    Returns:
    - new_price: adjusted price, bounded between min and max
    """
    if capacity == 0:
        return prev_price  # avoid division by zero

    # Calculate price change
    change = ALPHA * (occupancy / capacity)
    new_price = prev_price + change

    # Bound price within min and max
    new_price = max(min(new_price, MAX_PRICE), MIN_PRICE)
    return round(new_price, 2)


# Apply Baseline Pricing to All Lots

In [27]:
# Constants
BASE_PRICE = 10

# Example baseline price function from project description
def baseline_price(prev_price, occupancy, capacity, alpha=0.5):
    # Linear increase proportional to occupancy ratio
    price_increase = alpha * (occupancy / capacity) if capacity > 0 else 0
    return prev_price + price_increase

# If your dataframe is named lot_info, copy it to df
df = lot_info.copy()

# Add baseline_price column with initial base price
df_model1 = df.copy()
df_model1['baseline_price'] = BASE_PRICE

# Sort by ID and timestamp for simulation
df_model1.sort_values(by=['ID', 'timestamp'], inplace=True)

# Track last price per lot
lot_prices = {}

# Update prices row-wise
for i, row in df_model1.iterrows():
    lot_id = row['ID']
    prev_price = lot_prices.get(lot_id, BASE_PRICE)

    new_price = baseline_price(
        prev_price=prev_price,
        occupancy=row['Occupancy'],
        capacity=row['Capacity']
    )

    df_model1.at[i, 'baseline_price'] = new_price
    lot_prices[lot_id] = new_price


/tmp/ipython-input-27-1947571013.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '10.052859618717504' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_model1.at[i, 'baseline_price'] = new_price


# Visualize Baseline Price for a Sample Parking Lot

In [28]:
# 🎯 Choose one parking lot (first one)
sample_lot = df_model1['ID'].unique()[0]

# 📊 Filter data for that lot
sample_df = df_model1[df_model1['ID'] == sample_lot]

# 📦 Prepare Bokeh data source
source = ColumnDataSource(data={
    'timestamp': pd.to_datetime(sample_df['timestamp']),
    'price': sample_df['baseline_price']
})

# 🖼️ Create Bokeh plot (with updated method and dimensions)
p = figure(
    x_axis_type='datetime',
    title=f"Model 1: Baseline Price Over Time — Lot {sample_lot}",
    width=800, height=400
)
p.line('timestamp', 'price', source=source, line_width=2, color='navy')
p.scatter('timestamp', 'price', source=source, size=5, color='navy', alpha=0.5)

p.xaxis.axis_label = "Timestamp"
p.yaxis.axis_label = "Price ($)"

# 🔍 Show the plot
show(p)


## ✅ Model 2: Demand-Based Pricing — Summary

This model enhances the baseline by making the price sensitive to **real-world demand factors**:

### 📈 Demand Function
\[
\text{Demand} = \alpha \cdot \left(\frac{\text{Occupancy}}{\text{Capacity}}\right)
+ \beta \cdot \text{QueueLength}
- \gamma \cdot \text{Traffic}
+ \delta \cdot \text{IsSpecialDay}
+ \epsilon \cdot \text{VehicleWeight}
\]

### 🔧 Features Used:
- **Occupancy & Capacity**: Measures current space availability
- **Queue Length**: Higher queue → higher demand
- **Traffic Score**: Penalizes areas with high congestion
- **Special Day**: Boosts demand on holidays/events
- **Vehicle Type**: Prioritizes heavier vehicles like trucks (higher price potential)

### 🧮 Pricing Formula:
\[
\text{Price} = \text{BasePrice} \cdot \left(1 + \lambda \cdot \text{NormalizedDemand}\right)
\]

- Bounded between $5 and $20
- Smooth and explainable changes
- More sensitive to real-time conditions than Model 1




# Set Model Parameters

In [29]:
# 🔧 Demand Function Weights (tunable)
ALPHA_2 = 1.0    # Occupancy factor
BETA = 0.5       # Queue length factor
GAMMA = 0.3      # Traffic penalty
DELTA = 1.0      # Special day boost
EPSILON = 1.5    # Vehicle type importance
LAMBDA = 0.5     # Demand multiplier in final pricing

# 💵 Pricing bounds
MIN_PRICE_2 = 5.0
MAX_PRICE_2 = 20.0


# Define Vehicle Type Weights

In [30]:
# 🚗 Vehicle type weights
vehicle_weights = {
    'Car': 1.0,
    'Bike': 0.7,
    'Truck': 1.5
}

# 🧠 Apply weights safely and fill missing values
df_model2 = df.copy()
df_model2['vehicle_weight'] = df_model2['VehicleType'].map(vehicle_weights)
df_model2['vehicle_weight'] = df_model2['vehicle_weight'].fillna(1.0)  # safe assignment


# Convert Traffic to Numeric Values

In [31]:
# 🚦 Traffic level mapping (more traffic = more penalty)
traffic_mapping = {
    'Low': 1,
    'Medium': 2,
    'High': 3
}

# 🔄 Map traffic levels
df_model2['traffic_score'] = df_model2['TrafficConditionNearby'].map(traffic_mapping)

# 🧼 Replace missing or unknown values with average score
df_model2['traffic_score'] = df_model2['traffic_score'].fillna(2)


# Compute Demand Score

In [32]:
# 🧮 Demand score calculation for each row
df_model2['demand_score'] = (
    ALPHA_2 * (df_model2['Occupancy'] / df_model2['Capacity']) +
    BETA * df_model2['QueueLength'] -
    GAMMA * df_model2['traffic_score'] +
    DELTA * df_model2['IsSpecialDay'] +
    EPSILON * df_model2['vehicle_weight']
)


# Normalize Demand Score

In [33]:
# 📏 Normalize demand using Min-Max Scaling
min_demand = df_model2['demand_score'].min()
max_demand = df_model2['demand_score'].max()

df_model2['normalized_demand'] = (
    (df_model2['demand_score'] - min_demand) / (max_demand - min_demand)
)


#  Calculate Final Price from Demand

In [34]:
# 💰 Adjust price based on normalized demand
df_model2['demand_price'] = BASE_PRICE * (1 + LAMBDA * df_model2['normalized_demand'])

# ⛔ Enforce min and max price limits
df_model2['demand_price'] = df_model2['demand_price'].clip(lower=MIN_PRICE_2, upper=MAX_PRICE_2)

# 🎯 Round to 2 decimal places for clean pricing
df_model2['demand_price'] = df_model2['demand_price'].round(2)


# Visualize Demand-Based Pricing with Bokeh

In [35]:
# 🎯 Select same parking lot for comparison
sample_lot = df_model2['ID'].unique()[0]
sample_df_2 = df_model2[df_model2['ID'] == sample_lot]

# 🗃️ Prepare Bokeh data source
source2 = ColumnDataSource(data={
    'timestamp': pd.to_datetime(sample_df_2['timestamp']),
    'price': sample_df_2['demand_price']
})

# 📈 Bokeh plot for Model 2
p2 = figure(
    x_axis_type='datetime',
    title=f"Model 2: Demand-Based Price Over Time — Lot {sample_lot}",
    width=800, height=400
)
p2.line('timestamp', 'price', source=source2, line_width=2, color='green')
p2.scatter('timestamp', 'price', source=source2, size=5, color='green', alpha=0.5)

p2.xaxis.axis_label = "Timestamp"
p2.yaxis.axis_label = "Price ($)"

# 🔍 Show plot
show(p2)


## ✅ Model 3: Competitive Pricing — Summary

This final model introduces **market competition awareness** using geographic proximity.

### 💡 Key Idea:
A parking lot shouldn't price itself blindly — it should **react to competitor prices** nearby.

---

### 🔍 How It Works:

1. **Nearby Lots Detection**:
   - Calculated using the **Haversine distance**.
   - Any lot within **0.5 km** is considered a competitor.

2. **Competitive Adjustments**:
   - If a lot is **nearly full** and competitors are **cheaper** → **reduce price** slightly or suggest reroute.
   - If a lot has **low occupancy** and competitors are **more expensive** → **increase price**.
   - Otherwise, retain demand-based price.

3. **Final Formula**:
\[
\text{Final Price} =
\begin{cases}
P - 0.5, & \text{if overcrowded and competitors are cheaper} \\
P + 0.5, & \text{if underutilized and competitors are costlier} \\
P,       & \text{otherwise}
\end{cases}
\]

- Prices are still **bounded between $5 and $20**
- Ensures **realistic**, **explainable**, and **profitable** pricing

---

### 📊 Visualization:
Model 3 plot shows how the lot’s price responds to **market pressure** in real time.



# Haversine Distance Function

In [36]:
# 📍 Haversine distance to calculate proximity in kilometers
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return R * c


# Find Nearby Competitors

In [37]:
import numpy as np

# Extract all latitudes and longitudes
lot_ids = lot_info['ID'].values
latitudes = lot_info['Latitude'].values
longitudes = lot_info['Longitude'].values

# Convert degrees to radians
lat_r = np.radians(latitudes)
lon_r = np.radians(longitudes)

# Empty dictionary to store nearby lot IDs
nearby_lots_dict = {}

# Vectorized distance calculation
for i in range(len(lot_ids)):
    lat1, lon1 = lat_r[i], lon_r[i]
    lat2, lon2 = lat_r, lon_r

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    distances = 6371 * c  # Earth radius = 6371 km

    # Get nearby lot IDs (excluding self)
    nearby = lot_ids[(distances <= 0.5) & (distances > 0)]
    nearby_lots_dict[lot_ids[i]] = list(nearby)


# Apply Competitive Pricing

In [38]:
# 🧪 Work on a copy of Model 2 data
df_model3 = df_model2.copy()
df_model3['competitive_price'] = df_model3['demand_price']  # initialize

# 🔁 Sort data to simulate real-time processing
df_model3.sort_values(by=['ID', 'timestamp'], inplace=True)

# 🧠 Loop through rows and adjust price using nearby lots
for i, row in df_model3.iterrows():
    lot_id = row['ID']
    timestamp = row['timestamp']
    occupancy = row['Occupancy']
    capacity = row['Capacity']
    base_price = row['demand_price']

    # Find competitors for this lot
    nearby_ids = nearby_lots_dict.get(lot_id, [])
    if not nearby_ids:
        continue  # no competitors nearby

    # Get current prices of nearby lots at the same timestamp
    competitor_prices = df_model3[
        (df_model3['ID'].isin(nearby_ids)) &
        (df_model3['timestamp'] == timestamp)
    ]['demand_price'].values

    if len(competitor_prices) == 0:
        continue  # no competitor data at this timestamp

    avg_competitor_price = competitor_prices.mean()

    # 📈 Adjust based on competition and occupancy
    occupancy_ratio = occupancy / capacity if capacity != 0 else 1

    if occupancy_ratio >= 0.9 and base_price > avg_competitor_price:
        # Lot is nearly full, and competitors are cheaper
        new_price = base_price - 0.5
    elif occupancy_ratio <= 0.5 and base_price < avg_competitor_price:
        # Lot has space and can charge a bit more
        new_price = base_price + 0.5
    else:
        new_price = base_price

    # 📏 Keep price bounded
    df_model3.at[i, 'competitive_price'] = round(
        max(min(new_price, MAX_PRICE_2), MIN_PRICE_2), 2
    )


# Visualize Competitive Price Over Time

In [39]:
# 🎯 Choose the same sample lot as before
sample_lot = df_model3['ID'].unique()[0]
sample_df_3 = df_model3[df_model3['ID'] == sample_lot]

# 🎨 Prepare Bokeh data source
source3 = ColumnDataSource(data={
    'timestamp': pd.to_datetime(sample_df_3['timestamp']),
    'price': sample_df_3['competitive_price']
})

# 📈 Bokeh plot for Model 3
p3 = figure(
    x_axis_type='datetime',
    title=f"Model 3: Competitive Price Over Time — Lot {sample_lot}",
    width=800, height=400
)
p3.line('timestamp', 'price', source=source3, line_width=2, color='crimson')
p3.scatter('timestamp', 'price', source=source3, size=5, color='crimson', alpha=0.5)

p3.xaxis.axis_label = "Timestamp"
p3.yaxis.axis_label = "Price ($)"

# 🧾 Show the plot
show(p3)
